In [1]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV, lang='english', check=False)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.28.7, server:0.29.5

In [2]:
from absbox.examples import mixedAsset_test01

#### import a mix-asset-type deal

Lets import test case , which has two pools ,`PoolA` , `PoolB`, which holds `Morgage` and `Loan` respectively

In [3]:
mixedAsset_test01.pool

{'PoolA': {'assets': [['Mortgage',
    {'originBalance': 2200,
     'originRate': ['fix', 0.045],
     'originTerm': 30,
     'freq': 'Monthly',
     'type': 'Level',
     'originDate': '2021-02-01'},
    {'currentBalance': 2200,
     'currentRate': 0.08,
     'remainTerm': 30,
     'status': 'current'}]]},
 'PoolB': {'assets': [['Loan',
    {'originBalance': 80000,
     'originRate': ['floater',
      0.045,
      {'index': 'SOFR3M', 'spread': 0.01, 'reset': 'QuarterEnd'}],
     'originTerm': 60,
     'freq': 'Monthly',
     'type': 'i_p',
     'originDate': '2021-02-01'},
    {'currentBalance': 65000,
     'currentRate': 0.06,
     'remainTerm': 60,
     'status': 'Current'}]]}}

In [41]:
inst1 = ["Installment"
  ,{"originBalance": 1200.0
  ,"feeRate": ["fix",0.01]
  ,"originTerm": 12
  ,"freq": "Monthly"
  ,"type": ("PO_FirstN", 2)
  ,"originDate": "2022-01-01"}
  ,{"status": "Current"
      ,"currentBalance":1200
      ,"remainTerm":12}]


mixedAsset_test01.pool['PoolC'] = {"assets":[inst1]
                                  }

mixedAsset_test01.pool

{'PoolA': {'assets': [['Mortgage',
    {'originBalance': 2200,
     'originRate': ['fix', 0.045],
     'originTerm': 30,
     'freq': 'Monthly',
     'type': 'Level',
     'originDate': '2021-02-01'},
    {'currentBalance': 2200,
     'currentRate': 0.08,
     'remainTerm': 30,
     'status': 'current'}]]},
 'PoolB': {'assets': [['Loan',
    {'originBalance': 80000,
     'originRate': ['floater',
      0.045,
      {'index': 'SOFR3M', 'spread': 0.01, 'reset': 'QuarterEnd'}],
     'originTerm': 60,
     'freq': 'Monthly',
     'type': 'i_p',
     'originDate': '2021-02-01'},
    {'currentBalance': 65000,
     'currentRate': 0.06,
     'remainTerm': 60,
     'status': 'Current'}]]},
 'PoolC': {'assets': [['Installment',
    {'originBalance': 1200.0,
     'feeRate': ['fix', 0.01],
     'originTerm': 12,
     'freq': 'Monthly',
     'type': ('PO_FirstN', 2),
     'originDate': '2022-01-01'},
    {'status': 'Current', 'currentBalance': 1200, 'remainTerm': 12}]]}}

#### Set assumption by Pool Name

In [42]:
r = localAPI.run(mixedAsset_test01
                 ,poolAssump = ("ByName"
                                ,{"PoolA":
                                      (("Mortgage",{"CDR":0.02} ,None, None, None)
                                       ,None
                                       ,None)}
                                )
                 ,runAssump = [("interest",("LIBOR6M",0.04)
                                           ,("SOFR3M",0.04))
                               ,("inspect",("MonthEnd",("poolBalance","PoolB"))
                                          ,("MonthEnd",("poolBalance","PoolA"))
                                )]
                 ,read=True)

Warning Message from server:
Bond B is not paid off
Fee serviceFee is not paid off
Fee serviceFee2 is not paid off
Account acc02 has cash to be distributed

In [33]:
from lenses import lens
import pandas as pd

In [43]:
def readPoolsCf(pMap) -> pd.DataFrame:
    pNames = pMap & lens.Keys().collect()
    pFlows = pMap & lens.Values().collect()
    pColumns = pMap & lens.Values().F(lambda x:x.columns.to_list()).collect()
    
    headers = tz.concat([  [ (k,c) for c in cs]  for k,cs in zip(pNames,pColumns)])
    
    headerIndex = pd.MultiIndex.from_tuples(headers, names=["Pool", "Field"])

    df = pd.concat(pFlows,axis=1)
    df.columns = headerIndex
    return df

In [45]:
readPoolsCf(r['pool']['flow'])['PoolC']

Field,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,
2021-07-15,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
2021-07-31,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
2021-08-31,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-30,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
2021-10-31,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-30,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-31,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-31,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
2022-02-28,1100.0,100.0,0.0,0.0,0.0,0.0,0.0,0.01,100.0,0.0,0.0,0.0,0.0,0.0
